""" This notebook describes how one can use pickle library to save files so that the TensorFlow kernel can be used with Desi condition_spectra function. 

Saving files like this has two majpr advantages:

 (1) It saves a lot of time as one does not have to run the condition_spectra function everytime before training the network.
 
 (2) Since the DESI kernel has very limited compatability with TF and Keras. This allows the users to bypass that lack of cohesion and save files using the functions in DESI kernel and use TF on a different notebook or a seperate notebook without needing the functions of DESI Kernel"""

##### (1) Run the condition_spectra function on DESI Kernel to get the desired conditioned spectra of SN and Hosts

In [ ]:
from desispec.io import read_spectra
from desitrip.preproc import rebin_flux, rescale_flux

from glob import glob
import glob
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy.table import Table

import os
import platform

mpl.rc('font', size=14)

In [ ]:
def condition_spectra(coadd_files, truth_files):
    """Read DESI spectra, rebin to a subsampled logarithmic wavelength grid, and rescale.
    
    Parameters
    ----------
    coadd_files : list or ndarray
        List of FITS files on disk with DESI spectra.
    truth_files : list or ndarray
        Truth files.
    
    Returns
    -------
    fluxes : ndarray
        Array of fluxes rebinned to a logarithmic wavelength grid.
    """
    fluxes = None
    
    for cf, tf in zip(coadd_files, truth_files):
        spectra = read_spectra(cf)
        wave = spectra.wave['brz']
        flux = spectra.flux['brz']
        ivar = spectra.ivar['brz']
        
        truth = Table.read(tf, 'TRUTH')
        truez = truth['TRUEZ']
#         uid = truth ['TARGETID']
#         # Pre-condition: remove spectra with NaNs and zero flux values.
#         mask = np.isnan(flux).any(axis=1) | (np.count_nonzero(flux, axis=1) == 0)
#         mask_idx = np.argwhere(mask)
#         flux = np.delete(flux, mask_idx, axis=0)
#         ivar = np.delete(ivar, mask_idx, axis=0)

        # Rebin and rescale fluxes so that each is normalized between 0 and 1.
        rewave, reflux, reivar = rebin_flux(wave, flux, ivar, truez, minwave=2500., maxwave=9500., nbins=150, log=True, clip=True)
        rsflux = rescale_flux(reflux)

        if fluxes is None:
            fluxes = rsflux
        else:
            fluxes = np.concatenate((fluxes, rsflux))
    
    return fluxes

In [ ]:
snia_truth = np.sort(glob.glob((r'/global/project/projectdirs/desi/science/td/sim/bgs/180s/snia_airmass2*/*truth.fits')))
snia_coadd = np.sort(glob.glob((r'/global/project/projectdirs/desi/science/td/sim/bgs/180s/snia_airmass2*/*coadd.fits')))
snia_flux  = condition_spectra(snia_coadd, snia_truth)

In [ ]:
host_truth = np.sort(glob.glob((r'/global/project/projectdirs/desi/science/td/sim/bgs/180s/hosts_airmass2*/*truth.fits')))
host_files= np.sort(glob.glob((r'/global/project/projectdirs/desi/science/td/sim/bgs/180s/hosts_airmass2*/*coadd.fits')))
host_flux  = condition_spectra(host_files, host_truth)

## Saving the files:
- After you have saved the spectra flux then we can now go on to using pickle to save the output
- In this example notebook, I show how to save flux data using snia_flux, host_flux a np list that we obtained after using the condition_spectra function as shown above.

- wb stands for 'write binary'; it bascially encrypts your data as binary.
- In the first example belpow I am saving snia_flux from above as snia_flux_airmass2.data 
- .data specifies the extentsion and is needed

### General structure for saving a file to_be_saved as new_saved.data :

- address: address of the directory where you want to save your file

<code>
with open (r'address/ new_saved.data', 'wb') as whatever:
    pickle.dump(to_be_saved, whatever)
</code>

In [ ]:
import pickle

with open(r'/global/homes/v/vashisth/Binary/snia_flux_airmass2.data', 'wb') as f:
    pickle.dump(snia_flux, f) 
    
with open(r'/global/homes/v/vashisth/Binary/host_flux_airmass2.data', 'wb') as f:
    pickle.dump(host_flux, f)

## Loading pickled files
- You can load the files after changing the kernel of the notebook or you can simply make a new notebook where you want to use <b>Tensorflow or anything that is not compatible with DESI Kernel</b>.

### General structure for loading a file new_saved.data as to_be_used :
- Note: the pickled file (new_saved.data) needs to be in the same folder as where you want to it

<code>
with open (r'new_saved.data', 'rb') as whatever:
    to_be_used= pickle.load(whatever)
</code>

In [ ]:
import pickle 

with open('host_flux_airmass2.data', 'rb') as filehandle:
    host_flux = pickle.load(filehandle)

with open('snia_flux_airmass2.data', 'rb') as filehandle:
    # read the data as binary data stream
    snia_flux = pickle.load(filehandle)

### <b> I hope this was useful !!!</b>